<a href="https://colab.research.google.com/github/imhyunho99/2023-1--Deaplearning_Framework/blob/main/MNIST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.nn.init as init
import torchvision
from torchvision import transforms, datasets
%matplotlib inline

In [ ]:
DEVICE = torch.device('cuda:0') if torch.cuda.is_available() else torch.device('cpu')
# if torch.cuda.is_available():
#     DEVICE = torch.device('cuda:0')
# else:
#     DEVICE = torch.device('cpu')
print('Using PyTorch version: ', torch.__version__, 'Device: ', DEVICE)

In [ ]:
BATCH_SIZE = 32

train_dataset = datasets.MNIST(
    root="../data/MNIST",
    train=True,
    download=True,
    transform=transforms.ToTensor()
)
test_dataset = datasets.MNIST(
    root="../data/MNIST",
    train=False,
    download=True,
    transform=transforms.ToTensor()
)
train_loader = torch.utils.data.DataLoader(
    dataset=train_dataset,
    batch_size=BATCH_SIZE,
    shuffle=True
)
test_loader = torch.utils.data.DataLoader(
    dataset=test_dataset,
    batch_size=BATCH_SIZE,
    shuffle=False
)

In [ ]:
for (images, labels) in train_loader:
    print('images: ', images.size(), 'type: ', images.type())
    print('labels: ', labels.size(), 'type: ', labels.type())
    break

In [ ]:
images, labels = next(iter(train_loader))
print('images: ', images.size(), 'type: ', images.type())
print('labels: ', labels.size(), 'type: ', labels.type())

In [ ]:
idx = 7
img = np.squeeze(images[idx].numpy())

fig = plt.figure(figsize = (12,12)) 
ax = fig.add_subplot(111)
ax.imshow(img, cmap='gray')
width, height = img.shape
thresh = img.max()/2.5
for x in range(width):
    for y in range(height):
        val = round(img[x][y],2) if img[x][y] !=0 else 0
        ax.annotate(str(val), xy=(y,x),
                    horizontalalignment='center',
                    verticalalignment='center',
                    color='white' if img[x][y]<thresh else 'black')

In [ ]:
idx = 2
plt.imshow(np.squeeze(images[idx]), cmap='gray')
plt.title(str(labels[idx].item()))

In [ ]:
fig = plt.figure(figsize=(20, 8))
for idx in np.arange(BATCH_SIZE):
    ax = fig.add_subplot(4, int(BATCH_SIZE/4), idx+1, xticks=[], yticks=[])
    ax.imshow(np.squeeze(images[idx]), cmap='gray')
    ax.set_title(str(labels[idx].item()))

In [ ]:
def imshow(img):
    npimg = img.numpy()
    plt.axis('off')
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()

In [ ]:
images, labels = next(iter(train_loader))
imshow(torchvision.utils.make_grid(images))

In [ ]:
class MLP(nn.Module):
    def __init__(self):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(28 * 28, 512)
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, 10)
        #fc정의

    def forward(self, x):
        x = x.view(-1, 28 * 28)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        x = F.relu(x)
        x = self.fc3(x)
        x = F.log_softmax(x, dim=1)
        return x
    #정의에 대한 그들의 Link 설정 784 => 512 => 256 => 10, 100개의 데이터라고 하면 그들은 각각에 영향을 주지 않고 독립적으로 연산.

In [ ]:
def weight_init(m):
    if isinstance(m, nn.Linear):
        init.xavier_uniform_(m.weight.data)

model = MLP().to(DEVICE)
model.apply(weight_init)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
loss_fn = nn.CrossEntropyLoss()

In [ ]:
def train(model, train_loader, optimizer):
    model.train()
    train_loss = 0
    correct = 0
    for batch_idx, (image, label) in enumerate(train_loader):
        image = image.to(DEVICE)
        label = label.to(DEVICE)
        optimizer.zero_grad() 
        output = model(image) 
        loss = loss_fn(output, label) 
        train_loss += loss_fn(output, label).item()
        prediction = output.max(1, keepdim=True)[1]
        correct += prediction.eq(label.view_as(prediction)).sum().item()
        loss.backward() 
        optimizer.step() 

    train_loss /= len(train_loader.dataset)
    train_accuracy = 100. * correct / len(train_loader.dataset)
  
    return train_loss, train_accuracy

In [ ]:
def evaluate(model, test_loader):
    model.eval()
    test_loss = 0
    correct = 0

    with torch.no_grad():#평가이기 때문에 grad는 사용하지 않는다. 속도^
        for image, label in test_loader:
            image = image.to(DEVICE)
            label = label.to(DEVICE)
            output = model(image)#output 중 가장 큰 확률을 객체라고 생각한다.
            test_loss += loss_fn(output, label).item()
            prediction = output.max(1, keepdim=True)[1]
            correct += prediction.eq(label.view_as(prediction)).sum().item()
    
    test_loss /= len(test_loader.dataset)
    test_accuracy = 100. * correct / len(test_loader.dataset)
    return test_loss, test_accuracy#loss and eval return

In [ ]:
EPOCHS = 30
for epoch in range(1, EPOCHS + 1):
    train_loss,train_accuracy = train(model, train_loader, optimizer)
    test_loss, test_accuracy = evaluate(model, test_loader)
    print("[EPOCH: {}], \tTrain Loss: {:.4f}, \tTrain Accuracy: {:.2f}%, \tTest Loss: {:.4f}, \tTest Accuracy: {:.2f} %".format(
        epoch, train_loss, train_accuracy, test_loss, test_accuracy
    ))

In [ ]:
images, labels = next(iter(test_loader))
images = images.to(DEVICE)
labels = labels.to(DEVICE)

output = model(images)
_, preds = torch.max(output, 1)
images = images.cpu().numpy()

# plot the images in the batch, along with predicted and true labels
fig = plt.figure(figsize=(20, 8))
for idx in np.arange(BATCH_SIZE):
    ax = fig.add_subplot(4, int(BATCH_SIZE/4), idx+1, xticks=[], yticks=[])
    ax.imshow(np.squeeze(images[idx]), cmap='gray')
    ax.set_title("{} ({})".format(str(preds[idx].item()), str(labels[idx].item())),#정답과 예측값값
                 color=("green" if preds[idx]==labels[idx] else "red"))